IMDB movie review classification problem: sequence to class

## Pre process the data

In [13]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000
maxlen = 500
batch_size = 32

print('Loading data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(
     num_words=max_features)
print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')



Loading data...
25000 train sequences
25000 test sequences


By downloading the data, we get 250000 examples of movie reviews but each example as a differente size, so we use sequence.pad_sequences in order to obtain all examples the same size. The input to the network has the size (sequences, timesteps,features), where ``sequences`` is the number os examples, `timesteps` is the number of measurments obtained in each sequence and `features` is the number os variables in each timestep.

In [20]:
print('Pad sequences (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

Pad sequences (samples x time)
input_train shape: (25000, 500)
input_test shape: (25000, 500)


In [6]:
from keras.layers import Dense, Embedding, SimpleRNN
from keras.models import Sequential

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 14s 685us/step - loss: 0.5870 - acc: 0.6724 - val_loss: 0.5169 - val_acc: 0.7508
Epoch 2/10
20000/20000 [==============================] - 14s 708us/step - loss: 0.3689 - acc: 0.8488 - val_loss: 0.4033 - val_acc: 0.8228
Epoch 3/10
20000/20000 [==============================] - 14s 696us/step - loss: 0.2933 - acc: 0.8868 - val_loss: 0.3214 - val_acc: 0.8758
Epoch 4/10
20000/20000 [==============================] - 14s 703us/step - loss: 0.2146 - acc: 0.9231 - val_loss: 0.3450 - val_acc: 0.8694
Epoch 5/10
20000/20000 [==============================] - 14s 693us/step - loss: 0.1490 - acc: 0.9470 - val_loss: 0.3580 - val_acc: 0.8704
Epoch 6/10
20000/20000 [==============================] - 14s 679us/step - loss: 0.1005 - acc: 0.9665 - val_loss: 0.3881 - val_acc: 0.8686
Epoch 7/10
20000/20000 [==============================] - 14s 684us/step - loss: 0.0637 - acc: 0.9804 - val_loss: 0.5

The first naive approach to this dataset got you to a test accuracy of 88%. Unfortunately, this small recurrent network doesn’t perform well compared to this baseline (only 85% validation accuracy). Part of the problem is that your inputs only consider the first 500 words, rather than full sequences—hence, the RNN has access to less information than the earlier baseline model. The remainder of the problem is that SimpleRNN isn’t good at processing long sequences, such as text. Other types of recurrent layers perform much better. 

Also, in this example we are only trying to predict the class of each movie review, we are not trying to predict any value into the future.